# Accomodations for Caregivers in Bari

<div style="text-align: justify"> 
    
## An urban inquiry to provide a more comfortable stay to relatives and caregivers accompanying hospitalized patients 

</div>





## Table of Contents


* [Background](#Background)
* [Aim](#Aim)
* [Methods](#Methods)
* [Results](#Results)
* [Discussion](#Discussion)
* [Conclusions](#Conclusions)





#### Background 

Hospitals in Bari are the reference for good medical care in Southern Italy. Every year thousands of patients from nearby regions, together with their family and/or carigivers, move to Bari to undergo surgery or third level medical treatment requiring short to mid-term hospitalization. Nevertheless not every hospital neighborhood is capable of providing a comfortable stay to patients' relatives and/or caregivers during hospitalization becaouse of the lack of hotels and cheap food places. 





#### Aim
The Mayor of Bari requested an urban inquiry trying to unveil whether hospital neighborhoods require new food places and/or accomodations to improve the stay of those accompanying patients during hospitaliation .





#### Methods

##### Python Modules and Packages
The subsequent Python modules and packages were used throughout the project.


In [1]:
import requests
!pip install wget
import wget
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
!pip install geopy
!pip install folium
import folium
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

##### Data Collection and Cleaning
A database containing hospitals in Bari was used (freely accessible from https://github.com/drvincentvenus/My-Repository/blob/master/barihospital.xls). The latter database were cleaned from useless variables. Hospital coordinates were then used to acquire the geolocation of neighborhood services (i.e. <= 1 km from each hospital) through FOURSQUARE API. Data were then grouped by different hospitals and the total number of food places (i.e. Bars, Cafés, Bakeries, Restaurants and Pizza Places) and accomodations (i.e Hotels and Hostels) was then calculated.


##### Clusterization and Mapping
KMeans Clusterization algorithm was then used selecting geospacial coordinates, and the total numaber of accomodations and food places in the urban zones of each hospitals. Geopy webservice was used to map the identified clusters.



In [13]:
url= 'https://github.com/drvincentvenus/My-Repository/blob/master/barihospital.xls?raw=true'

df1=pd.read_excel(url)

df1

,Denominazione ASL territoriale,Denominazione struttura,Indirizzo,CAP,Sigla provincia,Codice Comune,Natura giuridica,lat,long
0,ASL BA,CASA DI CURA SANTA MARIA,VIA A. DE FERRARIIS 18/D,70124,BA,72006,Privata,41.108405,16.861457
1,ASL BA,CASA DI CURA ANTHEA,VIA C. ROSALBA 35/37,70124,BA,72006,Privata,41.104302,16.857360
2,ASL BA,C.B.H. MATER DEI HOSPITAL,"VIA HAHNEMANN, 10",70125,BA,72006,Privata,41.097925,16.885576
3,ASL BA,OSPEDALE BARI 'SAN PAOLO',VIA CAPOSCARDICCHIO S.N.,70123,BA,72006,Pubblica,41.119219,16.777129
4,ASL BA,OSPEDALE BARI 'DI VENERE' (TRIGGIANO),VIA OSPEDALE DI VENERE,70012,BA,72006,Pubblica,41.076839,16.863725
5,ASL BA,ISTITUTO TUMORI GIOVANNI PAOLO II,VIALE ORAZIO FLACCO N.65,70126,BA,72006,Pubblica,41.106603,16.858983
6,ASL BA,AO UNIV. CONS. POLICLINICO BARI,P.zza G. Cesare 11,70124,BA,72006,Pubblica,41.112282,16.862638


In [23]:
df1.rename(columns={'Denominazione ASL territoriale': 'ASL', 'CAP': 'Postal Code', 'Denominazione struttura': 'Hospital', 'Indirizzo': 'Address'}, inplace = True)
df1.drop(['Sigla provincia', 'Codice Comune', 'Natura giuridica', 'ASL'], axis = 1, inplace = True)


df1

,Hospital,Address,Postal Code,lat,long
0,CASA DI CURA SANTA MARIA,VIA A. DE FERRARIIS 18/D,70124,41.108405,16.861457
1,CASA DI CURA ANTHEA,VIA C. ROSALBA 35/37,70124,41.104302,16.857360
2,C.B.H. MATER DEI HOSPITAL,"VIA HAHNEMANN, 10",70125,41.097925,16.885576
3,OSPEDALE BARI 'SAN PAOLO',VIA CAPOSCARDICCHIO S.N.,70123,41.119219,16.777129
4,OSPEDALE BARI 'DI VENERE' (TRIGGIANO),VIA OSPEDALE DI VENERE,70012,41.076839,16.863725
5,ISTITUTO TUMORI GIOVANNI PAOLO II,VIALE ORAZIO FLACCO N.65,70126,41.106603,16.858983
6,AO UNIV. CONS. POLICLINICO BARI,P.zza G. Cesare 11,70124,41.112282,16.862638


In [18]:
CLIENT_ID = 'H2MOKQYZANUNAVFHHFBPYFQJZM5XC03HCFAYVA0GXCJSUDXA' #
CLIENT_SECRET = 'CQC3MTGLDAV0SR3IYAOGM1PFPWEAQ5XUTIXBRXV3KQCJED2X' 
VERSION = '20201102'

In [21]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=1000
    LIMIT=1000
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Hospital', 
                  'Hospital Latitude', 
                  'Hospital Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [24]:
hospital_venues = getNearbyVenues(names = df1['Hospital'],
                                   latitudes=df1['lat'],
                                   longitudes=df1['long']
                                   )

CASA DI CURA SANTA MARIA
CASA DI CURA ANTHEA
C.B.H. MATER DEI HOSPITAL
OSPEDALE BARI 'SAN PAOLO'
OSPEDALE BARI 'DI VENERE'  (TRIGGIANO)
ISTITUTO TUMORI GIOVANNI PAOLO II
AO UNIV. CONS. POLICLINICO BARI


In [25]:
hospital_venues
hospital_venues ['Venue Category'].value_counts()

Pizza Place                 30
Pub                         19
Café                        14
Japanese Restaurant         10
Bar                         10
Hotel                        7
Bakery                       6
Fast Food Restaurant         5
Electronics Store            5
Cocktail Bar                 5
Chinese Restaurant           4
Ice Cream Shop               4
Market                       4
Outdoors & Recreation        4
Train Station                4
Butcher                      4
Gastropub                    4
Brewery                      3
Steakhouse                   3
Restaurant                   3
Sushi Restaurant             2
Park                         2
Movie Theater                2
Plaza                        2
Mobile Phone Shop            2
Sandwich Place               2
Gym                          2
Beer Garden                  2
Stadium                      1
Business Service             1
Accessories Store            1
Breakfast Spot               1
Noodle H

In [27]:
hospital_oneshot = pd.get_dummies(hospital_venues[['Venue Category']], prefix="", prefix_sep="")


In [28]:

hospital_oneshot.insert(loc=0, column='Hospital', value=hospital_venues['Hospital'])
hospital_oneshot.shape

(181, 46)

In [29]:
hospital_oneshot

,Hospital,Accessories Store,Bakery,Bar,Beer Garden,Breakfast Spot,Brewery,Burger Joint,Business Service,Butcher,...,Pub,Restaurant,Sandwich Place,Shopping Mall,Soccer Field,Stadium,Steakhouse,Sushi Restaurant,Theater,Train Station
0,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9,CASA DI CURA SANTA MARIA,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [125]:
hospital_grouped = hospital_oneshot.groupby('Hospital').sum()
hospital_grouped['Accomodation'] = hospital_grouped['Hotel'] + hospital_grouped['Hostel']
hospital_grouped['Everyday Food'] = hospital_grouped['Bakery'] + hospital_grouped['Bar']
+ hospital_grouped['Breakfast Spot'] + hospital_grouped['Restaurant'] +  hospital_grouped['Café'] + hospital_grouped['Pizza Place']
hospital_grouped

df2= hospital_grouped[['Accomodation','Everyday Food']]

df3= df1.join(df2, on='Hospital')
df3.drop(['Address', 'Postal Code'],1, inplace =True)

df4= df3.set_index(['Hospital'], 0)

df4.drop(['Hospital'],1, inplace= True)


In [126]:
# set number of clusters
kclusters = 4


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df4)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

df4.insert(0, 'Cluster Labels', kmeans.labels_ )

df4

,Cluster Labels,lat,long,Accomodation,Everyday Food
Hospital,,,,,
CASA DI CURA SANTA MARIA,2,41.108405,16.861457,1,4
CASA DI CURA ANTHEA,2,41.104302,16.857360,1,3
C.B.H. MATER DEI HOSPITAL,3,41.097925,16.885576,0,2
OSPEDALE BARI 'SAN PAOLO',0,41.119219,16.777129,0,1
OSPEDALE BARI 'DI VENERE' (TRIGGIANO),0,41.076839,16.863725,0,0
ISTITUTO TUMORI GIOVANNI PAOLO II,2,41.106603,16.858983,1,3
AO UNIV. CONS. POLICLINICO BARI,1,41.112282,16.862638,5,3


In [118]:
address = 'Bari, IT'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of BARI are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of BARI are 41.1257843, 16.8620293.


#### Results
In Figure 1 4 dofferent clusters have been mapped. In Table 1 the total number of accomodations and food places have been displayed.

In [209]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(df4['lat'], df4['long'], df4['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters # Figure 1 

In [207]:
df5 = df4.groupby('Cluster Labels').sum()
df5= df5.drop(['lat', 'long'], 1)

print('Table 1', df5)

Tabl 1                 Accomodation  Everyday Food
Cluster Labels                             
0                          0              1
1                          5              3
2                          3             10
3                          0              2


#### Discussion

Every year up to 4000 patients move to Bari from nearby cities and regions to undergo surgery or specific medical treatment. Although massive urbanization and investments in the last years, services for caregivers accompanying hospitalized patients still lack in several areas. In Bari there are 6 third-level hospitals. Despite that, because three of them are very close,  as consequence of Kmeans algorithm, 4 clustered neighborhood were identified (Figure 1). Of note, hospitals in clusters 1 and 2 have lot of accomodations and foodplaces in <= 1 km. This may be explained considering thei location next to city centre. Conversely neighborhoods in cluster 0 and 3 lack hotels and restaurants. In general the Mayor should promote useful commercial activities for patients' caregivers accompanying patients in such hospitals.

#### Conclusions

Hospital neighborhood in Cluster 0 urgently need new accomodations. Mayor of Bari may consider to give new licences for restaurants too.

Hospital neighborhood in Cluster 1-2 may offer a nice stay because of several accomodations and food places 

Hospital neighborhoods in Cluster 3 needs more accomodations. 



    
